In [8]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import pm4py
import sys
sys.path.append('../') 
from utils.utilities import Utilities
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import time
from IPython.display import clear_output
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter
from pm4py.algo.discovery.inductive import variants
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.objects.conversion.process_tree import converter as pt_converter
import logging





# Import data

In [9]:
file_path = '../data/PermitLog.xes'
log = pm4py.read_xes(file_path, progress=False)

parsing log, completed traces ::   0%|          | 0/7065 [00:00<?, ?it/s]

In [10]:
def filter_log_by_start_and_end(log, start_activity, end_activity):
    '''
    Filter Noisy Traces and keep only traces with the main start / end activities
    '''
    log_start_filtered = start_activities_filter.apply(log, {start_activity})
    log_filtered = end_activities_filter.apply(log_start_filtered, {end_activity})
    return log_filtered

log = filter_log_by_start_and_end(log, "Permit SUBMITTED by EMPLOYEE", "Payment Handled")

# Hypeopt

In [11]:
budget_time, start_time, best_model, best_score, best_parameters = None, None, None, None, None
def check_is_time_limit_reached():
    
    if budget_time == None:
        return
    
    if time.time() - start_time > budget_time:
        print("Time limit reached")
        raise SystemExit


def get_net(params):    
    model_name = params['model_name']
    del params["model_name"]

    if model_name == 'alpha':
        net, initial_marking, final_marking = alpha_miner.apply(log)
    elif model_name == 'alpha_plus':
        net, initial_marking, final_marking =  alpha_miner.apply(log, variant=alpha_miner.Variants.ALPHA_VERSION_PLUS)
    elif model_name == 'inductive':

        variant_name = params['variant']
        del params["variant"]
        
        if variant_name == "im":
            variant = variants.im
        elif variant_name == "imd":
            variant = variants.imd
        else:
            variant = variants.imf  # default fallback

        process_tree = inductive_miner.apply(log, params, variant=variant)
        net, initial_marking, final_marking = pt_converter.apply(process_tree, variant=pt_converter.Variants.TO_PETRI_NET)
    elif model_name == 'heuristic':
        net, initial_marking, final_marking = pm4py.discover_petri_net_heuristics(log, **params)
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return (net, initial_marking, final_marking)

def objective(params):
    params_copy = params.copy()
    global best_model, best_score, best_parameters
    check_is_time_limit_reached()
    score_type = params['score_type']
    del params["score_type"]

    model = get_net(params)
    
    if score_type == 'fitness':
        score = Utilities.fitness_score(log, model)
    elif score_type == 'mixed':
        score = Utilities.mixed_score(log, model)
    else:
        raise ValueError(f"Unknown score type: {score_type}")

    if not best_score or score > best_score:
        best_score = score
        best_model = model
        best_parameters = params_copy
        logging.info(f"New best score: {best_score}")
        logging.info(f"Best parameters: {params_copy}")
        fitness = Utilities.fitness_score(log, best_model)
        generalization = Utilities.generalization_score(log, best_model)
        simplicity = Utilities.simplicity_score(log, best_model)

        logging.info(f"  Fitness: {fitness}")
        logging.info(f"  Generalization: {generalization}")
        logging.info(f"  Simplicity: {simplicity}")
        
    return -score

#### Search space

In [12]:
def create_search_space(eval_func_name):
    search = [
        {
            "model_name": "alpha",
            "score_type": eval_func_name
        },
        {
            "model_name": "inductive",
            "score_type": eval_func_name,
            "variant": hp.choice("variant", ["im", "imd", "imf"]),
            "noise_threshold": hp.uniform("noise_threshold", 0.0, 0.5),
        },
        {
            "model_name": "heuristic",
            "score_type": eval_func_name,
            "dependency_threshold": hp.uniform("dependency_threshold", 0.0, 0.9),
            "and_threshold": hp.uniform("and_threshold", 0.0, 0.9),
            "loop_two_threshold": hp.uniform("loop_two_threshold", 0.0, 0.9),
        }
    ]
    return hp.choice("classifier_type", search)


## Fitness

In [13]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Set the log level
    format="%(asctime)s - %(message)s",  # Log format includes timestamp
    handlers=[
        logging.FileHandler("hyperopt_single_objective_optimization.log"),
        logging.StreamHandler()  # Log to console
    ]
)

In [14]:
budget_time = 60 * 60 # 60 minutes
trials = Trials()
search_space = create_search_space('fitness')
start_time = time.time()

try:
    best_hyperparams_fitness = fmin(fn=objective,
                            space=search_space,
                            algo=tpe.suggest,
                            max_evals=1000,
                            trials=trials)
except SystemExit:
    pass

logging.info("Best trial:")
logging.info("  Value: ", best_parameters)
fitness = Utilities.fitness_score(log, best_model)
generalization = Utilities.generalization_score(log, best_model)
simplicity = Utilities.simplicity_score(log, best_model)

elapsed_time = time.time() - start_time
logging.info("  Fitness: ", fitness)
logging.info("  Generalization: ", generalization)
logging.info("  Simplicity: ", simplicity)
logging.info("  Elapsed Time: ", elapsed_time, "s")


  0%|          | 0/1000 [00:00<?, ?trial/s, best loss=?]

2024-12-15 22:54:46,009 - build_posterior_wrapper took 0.000999 seconds
2024-12-15 22:54:46,011 - TPE using 0 trials


replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

2024-12-15 22:54:49,588 - New best score: 0.3899259740893715
2024-12-15 22:54:49,589 - Best parameters: {'model_name': 'alpha', 'score_type': 'fitness'}


replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

...

100%|██████████| 1000/1000 [47:59<00:00,  2.88s/trial, best loss: -1.0]

2024-12-15 23:42:45,243 - Best trial:
2024-12-15 23:42:45,244 -   Value: 


replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Noel Nathan\AppData\Roaming\

In [17]:
print(best_parameters)

{'model_name': 'inductive', 'noise_threshold': 0.4803000397767385, 'score_type': 'fitness', 'variant': 'imd'}


In [15]:
best_model_fitness = best_model

### Saving metrics for the test

In [ ]:

data = Utilities.evaluate_and_save(log, best_model_fitness, "hyperopt5min", "fitness")


NameError: name 'ghhjgj' is not defined

## Mixed Score

In [10]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Set the log level
    format="%(asctime)s - %(message)s",  # Log format includes timestamp
    handlers=[
        logging.FileHandler("hyperopt_multi_objective_optimization.log"),
        logging.StreamHandler()  # Log to console
    ]
)

In [11]:
best_model, best_score = None, None

budget_time = 60 * 60 # 60 minutes
trials = Trials()
search_space = create_search_space('mixed')
start_time = time.time()
try:
    best_hyperparams_mixed = fmin(fn=objective,
                            space=search_space,
                            algo=tpe.suggest,
                            max_evals=1000,
                            trials=trials)

except SystemExit:
    pass

logging.info("Best trial:")
logging.info("  Value: ", best_parameters)
fitness = Utilities.fitness_score(log, best_model)
generalization = Utilities.generalization_score(log, best_model)
simplicity = Utilities.simplicity_score(log, best_model)

elapsed_time = time.time() - start_time
logging.info("  Fitness: ", fitness)
logging.info("  Generalization: ", generalization)
logging.info("  Simplicity: ", simplicity)
logging.info("  Elapsed Time: ", elapsed_time, "s")

replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1020 [00:00<?, ?it/s]

  0%|          | 2/1000 [00:42<5:54:43, 21.33s/trial, best loss: -0.8140905856458556]


KeyboardInterrupt: 

In [10]:
best_model_mixed = best_model

### Saving metrics for the test

In [ ]:
data = Utilities.evaluate_and_save(log, best_model_mixed,"hyperopt5min", "mixed")